In [6]:
from encoder_model import EncoderDNN
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import accuracy_score
import torch as torch

import data_helper_413 as DataHelper
import model_413 as Model

os.environ["CUDA_VISIBLE_DEVICES"]='0'
from keras.backend.tensorflow_backend import set_session
config=tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction=0.9
set_session(tf.Session(config=config))


base_dir= os.getcwd()
test_csv_path=os.path.join(base_dir,'/home/aries/413/CNNLoc-Access/TestData.csv')
valid_csv_path=os.path.join(base_dir,'/home/aries/413/CNNLoc-Access/ValuationData.csv')
train_csv_path=os.path.join(base_dir,'/home/aries/413/CNNLoc-Access/TrainingData.csv')





Using TensorFlow backend.
/home/aries/miniconda3/envs/CNNLOC/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aries/miniconda3/envs/CNNLOC/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aries/miniconda3/envs/CNNLOC/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aries/minicond

In [44]:
(train_x, train_y), (valid_x, valid_y),(test_x,test_y) = DataHelper.load_data_all(train_csv_path, valid_csv_path,test_csv_path)
(train_x,train_y) = Model.filter_building(train_x,train_y,1)
(valid_x, valid_y) = Model.filter_building(valid_x, valid_y, 1)
nn_model = Model.NN()
nn_model._preprocess(train_x[:2000],train_y[:2000],valid_x[:400],valid_y[:400])
nn_model._knn_process()

0.9965


In [25]:
def preprocess_data(data):
    # Initialize a list to store the preprocessed data
    preprocessed_data = []

    # Iterate over each sample in the batch
    for sample in data:
        # Find the indices and values of non-zero signal strengths
        non_zero_indices = np.nonzero(sample)[0]
        non_zero_values = sample[non_zero_indices]

        # Sort the indices by signal strength in descending order
        sorted_indices = non_zero_indices[np.argsort(-non_zero_values)]
        sorted_values = non_zero_values[np.argsort(-non_zero_values)]

        # Create a matrix for the sample where each row is a one-hot encoded vector
        # multiplied by the corresponding signal strength
        signal_strength_matrix = np.zeros((len(sorted_indices), 520))
        signal_strength_matrix[np.arange(len(sorted_indices)), sorted_indices] = sorted_values

        # Append the signal strength matrix to the preprocessed data
        preprocessed_data.append(signal_strength_matrix)

    # Determine the maximum number of non-zero values in a sample
    M = max(matrix.shape[0] for matrix in preprocessed_data)

    # Pad each signal strength matrix to have the same number of rows
    for i in range(len(preprocessed_data)):
        padding = M - preprocessed_data[i].shape[0]
        preprocessed_data[i] = np.pad(preprocessed_data[i], ((0, padding), (0, 0)), 'constant')

    # Stack the preprocessed data to form the final output
    preprocessed_data = np.stack(preprocessed_data)

    return preprocessed_data

In [35]:
from torch import nn

class LocalizationModel(nn.Module):
    def __init__(self, num_waps, embedding_dim, output_dim):
        super(LocalizationModel, self).__init__()
        self.embedding = nn.Embedding(num_waps, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, x, signal_strengths):
        # x is of shape (batch_size, M), where M is the maximum number of non-zero signal strengths in a sample
        # signal_strengths is of shape (batch_size, M), corresponding to the signal strengths of the WAPs in x

        # Embed the WAP indices
        x = self.embedding(x)  # shape: (batch_size, M, embedding_dim)

        # Multiply the embeddings by the signal strengths
        x = x * signal_strengths.unsqueeze(-1)  # shape: (batch_size, M, embedding_dim)

        # Sum across the signal strength dimension to get a single vector for each sample
        x = x.sum(dim=1)  # shape: (batch_size, embedding_dim)

        # Pass through the fully connected layer to get the output
        out = self.fc(x)  # shape: (batch_size, output_dim)

        return out

In [48]:
model = LocalizationModel(num_waps=520, embedding_dim=50, output_dim=2)

from torch.nn.functional import cosine_similarity

# Get the embeddings for WAPs 0 and 1
embedding_0 = model.embedding.weight.data[0]
embedding_1 = model.embedding.weight.data[1]

print(embedding_0)

# Compute the cosine similarity
similarity = cosine_similarity(embedding_0, embedding_1, dim=0)

print(similarity)

tensor([-1.2583,  0.8301,  0.6216,  0.3729,  0.5244, -0.1723,  0.1024,  0.3812,
         0.0656, -1.7183,  1.3714,  1.2918,  0.7563,  0.1842, -0.3303, -0.7168,
        -0.4953,  0.7723,  2.7056, -0.6074, -1.2438,  0.9854,  0.2585, -0.4099,
        -0.4263,  1.4122, -0.8190, -2.3250,  1.1550,  0.8452, -1.6340, -0.3426,
        -0.7726, -1.7961, -0.1756,  1.1305,  0.3386,  0.7110, -0.4861,  1.0858,
         0.0863,  0.7967, -1.0174, -0.6660, -0.1497,  1.2780,  0.2714,  1.0834,
        -0.5369,  0.5601])
tensor(0.2395)


In [54]:
from torch.utils.data import TensorDataset, DataLoader

print(nn_model.normalize_valid_x[0].max())

pre_x = preprocess_data(nn_model.normalize_x)
pre_valid_x = preprocess_data(nn_model.normalize_valid_x)
pre_y = nn_model.floorID_y
pre_valid_y = nn_model.floorID_valid_y

num_classes = 4

print(pre_x.shape)
print(pre_y.shape)

# Convert the datasets to PyTorch tensors
pre_x = torch.tensor(pre_x, dtype=torch.long)
pre_valid_x = torch.tensor(pre_valid_x, dtype=torch.long)
pre_y = torch.tensor(pre_y, dtype=torch.long)
pre_valid_y = torch.tensor(pre_valid_y, dtype=torch.long)

# Create TensorDatasets
train_dataset = TensorDataset(pre_x, pre_y)
valid_dataset = TensorDataset(pre_valid_x, pre_valid_y)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=batch_size)

0.10173535814711922
(2000, 48, 520)
(2000,)


TypeError: 'DataLoader' object does not support indexing

In [56]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss
import matplotlib.pyplot as plt

# Define the device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# Create the model
model = LocalizationModel(num_waps=520, embedding_dim=50, output_dim=num_classes).to(device)

# Define the loss function and optimizer
criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Define the number of epochs
num_epochs = 10

# Initialize lists to store losses and accuracies
train_losses, val_losses, train_accs, val_accs = [], [], [], []

for epoch in range(num_epochs):
    # Training
    train_loss, train_correct, total = 0, 0, 0
    model.train()
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        # Separate the signal strengths from the inputs
        signal_strengths = inputs.sum(dim=2)  # shape: (batch_size, M)
        inputs = (inputs > 0).long()  # shape: (batch_size, M, 520)

        optimizer.zero_grad()
        outputs = model(inputs, signal_strengths)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        train_correct += predicted.eq(targets).sum().item()

    train_losses.append(train_loss / len(train_loader))
    train_accs.append(train_correct / total)

    # Validation
    val_loss, val_correct, total = 0, 0, 0
    model.eval()
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.to(device), targets.to(device)

            # Separate the signal strengths from the inputs
            signal_strengths = inputs.sum(dim=2)  # shape: (batch_size, M)
            inputs = (inputs > 0).long()  # shape: (batch_size, M, 520)

            outputs = model(inputs, signal_strengths)
            loss = criterion(outputs, targets)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()

    val_losses.append(val_loss / len(val_loader))
    val_accs.append(val_correct / total)

    print(f'Epoch: {epoch+1}, Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accs[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accs[-1]:.4f}')
    
# Plotting
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train')
plt.plot(val_accs, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.